## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
from slugify import slugify
from datetime import datetime
import json

/Users/slee/.local/share/virtualenvs/california-coronavirus-scrapers-QTClc1WQ/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Download

Retrieve the data

In [4]:
POWER_BI_QUERY_URL = (
    "https://wabi-us-gov-iowa-api.analysis.usgovcloudapi.net/public/reports/querydata"
)
REQUEST_PARAMS = (("synchronous", "true"), ("Connection", "keep-alive"))
# REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"t1","Entity":"tab2_cities","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"cities"},"Name":"tab2_cities.cities"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"# Known Cases"}},"Function":0},"Name":"Sum(tab2_cities.# Known Cases)"},{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"% Known Cases"},"Name":"Sum(tab2_cities.% Known Cases)"},{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"% County Population"},"Name":"tab2_cities.% County Population"}],"OrderBy":[{"Direction":1,"Expression":{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t1"}},"Property":"# Known Cases"}},"Function":0}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3]}]},"DataReduction":{"DataVolume":3,"Primary":{"Window":{"Count":500}}},"Version":1},"ExecutionMetricsKind":1}}]},"CacheKey":"{\\"Commands\\":[{\\"SemanticQueryDataShapeCommand\\":{\\"Query\\":{\\"Version\\":2,\\"From\\":[{\\"Name\\":\\"t1\\",\\"Entity\\":\\"tab2_cities\\",\\"Type\\":0}],\\"Select\\":[{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"cities\\"},\\"Name\\":\\"tab2_cities.cities\\"},{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"# Known Cases\\"}},\\"Function\\":0},\\"Name\\":\\"Sum(tab2_cities.# Known Cases)\\"},{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"% Known Cases\\"},\\"Name\\":\\"Sum(tab2_cities.% Known Cases)\\"},{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"% County Population\\"},\\"Name\\":\\"tab2_cities.% County Population\\"}],\\"OrderBy\\":[{\\"Direction\\":1,\\"Expression\\":{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"t1\\"}},\\"Property\\":\\"# Known Cases\\"}},\\"Function\\":0}}}]},\\"Binding\\":{\\"Primary\\":{\\"Groupings\\":[{\\"Projections\\":[0,1,2,3]}]},\\"DataReduction\\":{\\"DataVolume\\":3,\\"Primary\\":{\\"Window\\":{\\"Count\\":500}}},\\"Version\\":1},\\"ExecutionMetricsKind\\":1}}]}","QueryId":"","ApplicationContext":{"DatasetId":"da83725f-fd62-480a-9a33-a73b06aa6bb1","Sources":[{"ReportId":"5663b2c5-89c2-4903-bb89-0086bbd23b8a","VisualId":"4dbc0385a9d366c66bd6"}]}}],"cancelQueries":[],"modelId":350069}'

In [5]:
REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"t","Entity":"tab2_zip","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"zips"},"Name":"tab2_zip.zips"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"# Known Cases"}},"Function":0},"Name":"Sum(tab2_zip.# Known Cases)"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"% Known Cases"}},"Function":0},"Name":"Sum(tab2_zip.% Known Cases)"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"% County Population"}},"Function":0},"Name":"Sum(tab2_zip.% County Population)"}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3]}]},"DataReduction":{"DataVolume":3,"Primary":{"Top":{"Count":30000}}},"Version":1,"Highlights":[{"Version":2,"From":[{"Name":"t","Entity":"tab2_zip","Type":0}],"Where":[{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"t"}},"Property":"zips"}}],"Values":[[{"Literal":{"Value":"\'95067\'"}}]]}}}]}]},"ExecutionMetricsKind":1}}]},"QueryId":"","ApplicationContext":{"DatasetId":"bd81ee59-dda3-44d6-a3a1-236bee18fe5d","Sources":[{"ReportId":"f129b0d3-3db0-46fd-b627-f4b0543ed028","VisualId":"b3260308f719b39b9973"}]}}],"cancelQueries":[],"modelId":369251}'

In [6]:
headers = {
    "Host": "wabi-us-gov-iowa-api.analysis.usgovcloudapi.net",
    "Connection": "keep-alive",
    "Content-Length": "2423",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
    "Accept": "application/json, text/plain, */*",
    "RequestId": "292cbfb4-7c1f-02ee-0494-d3d613eede46",  # change
    "DNT": "1",
    "X-PowerBI-ResourceKey": "f75b3578-0ecc-401c-9e6a-b79135c32922",  # change
    "Content-Type": "application/json;charset=UTF-8",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "ActivityId": "ae900a2e-d369-49d9-8f98-826c3af308a5",
    "Origin": "https://app.powerbigov.us",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Dest": "empty",
    "Referer": "https://app.powerbigov.us/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9,da;q=0.8",
}
json_data = json.loads(REQUEST_BODY)

In [7]:
response = requests.post(
    POWER_BI_QUERY_URL,
    params=REQUEST_PARAMS,
    headers=headers,
    json=json_data,
    verify=False,
)

/Users/slee/.local/share/virtualenvs/california-coronavirus-scrapers-QTClc1WQ/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wabi-us-gov-iowa-api.analysis.usgovcloudapi.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:
data = response.json()

In [9]:
timestamp = response.headers["Date"]

In [10]:
timestamp

'Tue, 19 Apr 2022 22:25:34 GMT'

In [11]:
ds = data["results"][0]["result"]["data"]["dsr"]["DS"][0]

In [12]:
ds

{'N': 'DS0',
 'PH': [{'DM0': [{'S': [{'N': 'G0', 'T': 1},
      {'N': 'M0', 'T': 4},
      {'N': 'H0', 'T': 4},
      {'N': 'M1', 'T': 3},
      {'N': 'H1', 'T': 3},
      {'N': 'M2', 'T': 3},
      {'N': 'H2', 'T': 3}],
     'C': ['95001', 128, 0.26, 0],
     'Ø': 84},
    {'C': ['95003', 3114, 6.44, 8.586215], 'R': 84},
    {'C': ['95005', 739, 1.53, 2.6118322], 'R': 84},
    {'C': ['95006', 875, 1.81, 3.2837968], 'R': 84},
    {'C': ['95007', 105, 0.22, 0.1275062], 'R': 84},
    {'C': ['95010', 1554, 3.21, 3.0192384], 'R': 84},
    {'C': ['95017', 79, 0.16, 0.2321704], 'R': 84},
    {'C': ['95018', 921, 1.9, 2.3322731], 'R': 84},
    {'C': ['95019', 2177, 4.5, 2.5119409], 'R': 84},
    {'C': ['95033', 362, 0.75, 3.2425448], 'R': 84},
    {'C': ['95041', 93, 0.19, 0.1486436], 'R': 84},
    {'C': ['95060', 6879, 14.22, 16.2965918], 'R': 84},
    {'C': ['95062', 5901, 12.19, 12.5245893], 'R': 84},
    {'C': ['95063', 102, 0.21, 0], 'R': 84},
    {'C': ['95064', 791, 1.63, 3.5749474], '

## Parse

In [13]:
rows = ds["PH"][0]["DM0"][1:]

In [14]:
rows

[{'C': ['95003', 3114, 6.44, 8.586215], 'R': 84},
 {'C': ['95005', 739, 1.53, 2.6118322], 'R': 84},
 {'C': ['95006', 875, 1.81, 3.2837968], 'R': 84},
 {'C': ['95007', 105, 0.22, 0.1275062], 'R': 84},
 {'C': ['95010', 1554, 3.21, 3.0192384], 'R': 84},
 {'C': ['95017', 79, 0.16, 0.2321704], 'R': 84},
 {'C': ['95018', 921, 1.9, 2.3322731], 'R': 84},
 {'C': ['95019', 2177, 4.5, 2.5119409], 'R': 84},
 {'C': ['95033', 362, 0.75, 3.2425448], 'R': 84},
 {'C': ['95041', 93, 0.19, 0.1486436], 'R': 84},
 {'C': ['95060', 6879, 14.22, 16.2965918], 'R': 84},
 {'C': ['95062', 5901, 12.19, 12.5245893], 'R': 84},
 {'C': ['95063', 102, 0.21, 0], 'R': 84},
 {'C': ['95064', 791, 1.63, 3.5749474], 'R': 84},
 {'C': ['95065', 1299, 2.68, 2.8948005], 'R': 84},
 {'C': ['95066', 1991, 4.11, 5.2301419], 'R': 84},
 {'C': ['95067', 49, 49, 0.1, 0.1, 0, 0]},
 {'C': ['95073', 1501, 3.1, 3.7546153], 'Ø': 84},
 {'C': ['95076', 19083, 39.44, 29.6281523], 'R': 84},
 {'C': ['Under investigation', 648, 1.34, 0], 'R': 84}]

In [15]:
confirmed_cases = []

for r in rows:
    confirmed_cases.append(r["C"][0:2])

In [16]:
confirmed_cases

[['95003', 3114],
 ['95005', 739],
 ['95006', 875],
 ['95007', 105],
 ['95010', 1554],
 ['95017', 79],
 ['95018', 921],
 ['95019', 2177],
 ['95033', 362],
 ['95041', 93],
 ['95060', 6879],
 ['95062', 5901],
 ['95063', 102],
 ['95064', 791],
 ['95065', 1299],
 ['95066', 1991],
 ['95067', 49],
 ['95073', 1501],
 ['95076', 19083],
 ['Under investigation', 648]]

In [17]:
df = pd.DataFrame(confirmed_cases, columns=["zip", "confirmed_cases"])
df.insert(0, "county", "Santa Cruz")

## Combine

Combine zips that overlap

In [18]:
pairs = {"95041": "95018", "95063": "95062", "95067": "95066"}

In [19]:
for k, v in pairs.items():
    df.loc[df.zip == v, "confirmed_cases"] = (
        df.loc[df.zip == v, "confirmed_cases"].item()
        + df.loc[df.zip == k, "confirmed_cases"].item()
    )

In [20]:
df

,county,zip,confirmed_cases
0,Santa Cruz,95003,3114
1,Santa Cruz,95005,739
2,Santa Cruz,95006,875
3,Santa Cruz,95007,105
4,Santa Cruz,95010,1554
5,Santa Cruz,95017,79
6,Santa Cruz,95018,1014
7,Santa Cruz,95019,2177
8,Santa Cruz,95033,362
9,Santa Cruz,95041,93


In [21]:
df = df[~df.zip.isin(pairs.keys())].copy()

In [22]:
df["county_date"] = pd.to_datetime(timestamp)

In [23]:
df["county_date"] = df["county_date"].dt.tz_convert("US/Pacific")

In [24]:
df["county_date"] = df["county_date"].dt.strftime("%Y-%m-%d")

In [25]:
df.rename(columns={"zip": "area"}, inplace=True)

In [26]:
pairs.keys()

dict_keys(['95041', '95063', '95067'])

## Vet

In [27]:
try:
    assert not len(df) > 17
except AssertionError:
    raise AssertionError("Santa Cruz' scraper has extra rows")

In [28]:
try:
    assert not len(df) < 17
except AssertionError:
    raise AssertionError("Santa Cruz' scraper is missing rows")

## Export

Set date

In [29]:
tz = pytz.timezone("America/Los_Angeles")

In [30]:
today = datetime.now(tz).date()

In [31]:
slug = "santa-cruz"

In [32]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [33]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [34]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [35]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [36]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)